## Set Libraries

In [1]:
import numpy as np
import xarray as xr
import rasterio
%matplotlib inline
from matplotlib.pyplot import *
from glob import glob
import os
import datetime
import glob
import pandas as pd
from rasterio_to_xarray import rasterio_to_xarray, xarray_to_rasterio

## Function to read in the MAIAC files into a data array

In [2]:
def maiac_file_to_da(filename):
    da = rasterio_to_xarray(filename)
    
    time_str = os.path.basename(filename)[17:-13]
    time_obj = datetime.datetime.strptime(time_str, '%Y%j%H%M')
    da.coords['time'] = time_obj
    
    return da

files = glob(r'D:\MAIAC_Data\Europe\h00v01\Projected\????\*_proj.tif')

In [ ]:
folders = glob.glob(r'D:\MAIAC_Data\Europe\h00v01\Projected\????')

In [ ]:
#folders = glob.glob(r'D:\Annies_Dissertation\MAIAC_Test\Projected\2014')

In [ ]:
folders

for folder in folders:
    files = glob.glob('{fname}\*_proj.tif'.format(fname=folder))
    print('Processing {fname}'.format(fname=folder))
    print(files)

## For loop to read MAIAC files in, year by year and create NetCDF files.

### AOT

In [ ]:
for folder in folders:
    files = glob.glob(r'{fname}\*_proj.tif'.format(fname=folder))
    list_of_das = map(maiac_file_to_da, files)
    MAIAC_AOT = xr.concat(list_of_das, 'time')
    reordered_MAIAC_AOT = MAIAC_AOT.isel(time=np.argsort(MAIAC_AOT.time))
    ds = reordered_MAIAC_AOT.to_dataset(name = 'data')
    ds.to_netcdf(r'D:\MAIAC_Data\Europe\h00v01\nc_files\{fol_name}_AOT.nc'.format(fol_name=os.path.basename(folder)))
    
    print('Processed {fname}'.format(fname=folder))

### PM2.5

Robin's stuff!

In [3]:
files = glob.glob(r'{fname}\*_PM25.tif'.format(fname=r'D:\MAIAC_Data\Europe\h00v01\Projected\2005'))

In [4]:
len(files)

2013

In [5]:
list_of_das = map(maiac_file_to_da, files)

In [6]:
MAIAC_AOT = xr.concat(list_of_das, 'time')

In [7]:
reordered_MAIAC_AOT = MAIAC_AOT.isel(time=np.argsort(MAIAC_AOT.time))

In [8]:
ds = reordered_MAIAC_AOT.to_dataset(name='data')

In [9]:
g = reordered_MAIAC_AOT.groupby('time.month')

In [10]:
ind = g.groups[1]

In [11]:
res = reordered_MAIAC_AOT.isel(time=ind)

In [12]:
folder = r'D:\MAIAC_Data\Europe\h00v01\Projected\2005'
for month, indices in g.groups.items():
    subset = reordered_MAIAC_AOT.isel(time=indices)
    sub_ds = subset.to_dataset(name = 'data')
    filename = r'D:\MAIAC_Data\Europe\h00v01\nc_files_new\{fol_name}_{month}_PM25.nc'.format(fol_name=os.path.basename(folder), month=month)
    print(filename)
    sub_ds.to_netcdf(filename)

D:\MAIAC_Data\Europe\h00v01\nc_files_new\2005_1_PM25.nc
D:\MAIAC_Data\Europe\h00v01\nc_files_new\2005_2_PM25.nc
D:\MAIAC_Data\Europe\h00v01\nc_files_new\2005_3_PM25.nc
D:\MAIAC_Data\Europe\h00v01\nc_files_new\2005_4_PM25.nc
D:\MAIAC_Data\Europe\h00v01\nc_files_new\2005_5_PM25.nc
D:\MAIAC_Data\Europe\h00v01\nc_files_new\2005_6_PM25.nc
D:\MAIAC_Data\Europe\h00v01\nc_files_new\2005_7_PM25.nc
D:\MAIAC_Data\Europe\h00v01\nc_files_new\2005_8_PM25.nc
D:\MAIAC_Data\Europe\h00v01\nc_files_new\2005_9_PM25.nc
D:\MAIAC_Data\Europe\h00v01\nc_files_new\2005_10_PM25.nc
D:\MAIAC_Data\Europe\h00v01\nc_files_new\2005_11_PM25.nc
D:\MAIAC_Data\Europe\h00v01\nc_files_new\2005_12_PM25.nc


End of Robin's stuff

In [ ]:
for folder in folders:
    files = glob.glob(r'{fname}\*_PM25.tif'.format(fname=folder))
    list_of_das = map(maiac_file_to_da, files)
    MAIAC_AOT = xr.concat(list_of_das, 'time')
    reordered_MAIAC_AOT = MAIAC_AOT.isel(time=np.argsort(MAIAC_AOT.time))
    ds = reordered_MAIAC_AOT.to_dataset(name = 'data')
    ds.to_netcdf(r'D:\MAIAC_Data\Europe\h00v01\nc_files\{fol_name}_PM25.nc'.format(fol_name=os.path.basename(folder)))
    
    print('Processed {fname}'.format(fname=folder))

In [ ]:
all_data = xr.open_mfdataset(r'D:\Annies_Dissertation\Methodology\Validation\AERONET\Test\*.nc', 
                                 chunks={'time':10}, concat_dim='time')
                                 
all_data = all_data['data']

In [ ]:
all_data = xr.open_mfdataset(r'D:\MAIAC_Data\Europe\h00v01\nc_files\*_PM25.nc', 
                                 chunks={'time':10}, concat_dim='time')
                                 
all_data = all_data['data']

## For loop to read the MAIAC data in and extract AOT for a specific pixel (Chilbolton) and export as csv

In [ ]:
for folder in folders:
    files = glob.glob(r'{fname}\*_proj.tif'.format(fname=folder))
    list_of_das = map(maiac_file_to_da, files)
    MAIAC_AOT = xr.concat(list_of_das, 'time')
    MAIAC_AOT.attrs.clear()
    reordered_MAIAC_AOT = MAIAC_AOT.isel(time=np.argsort(MAIAC_AOT.time))
    Measurements = reordered_MAIAC_AOT.isel(x=1103, y=1027).dropna('time').to_dataframe(name='MAIAC_AOT')
    Measurements.to_csv(r'D:\Annies_Dissertation\Methodology\Validation\AERONET\Test\test.csv', mode='a')
    print('Processed {fname}'.format(fname=folder))

Measurements.to_csv(r'D:\Annies_Dissertation\Methodology\Validation\AERONET\Test\Alldata.csv')

In [ ]:
    doc = open(r'D:\Annies_Dissertation\Methodology\Validation\AERONET\Test\Alldata.csv', 'a')
    doc.write(Measurements)
    doc.close()